OVERVIEW OF BELOW CODE:
- The below code seeks to use the cleaned dataframe from Script 00 (our input) to regrade the test to accomodate for giving partial credit on short answer questions. The cleaning was not done to the extent that was needed for this part, so some more very basic cleaning is conducted. Because one of our goals is to see if there exists a correlation between getting certain questions correct and getting a higher score, this code will help us in our OLS regression. 
- The final output is a .csv file that contains the question number as a column name and the values are the scores. We can use a simple merging function to recombine this back with the original data. 

DEFINING THE FUNCTION:
- Function name: regrading
   - Purpose: Because the dataset 1) does not label the scores by question for each respondent, and we're hoping to see if there is a correlation between getting certain scores right and scoring higher on the exam and 2) because we are unsure how the short answer sections are graded, so we regraded such that they would be awarded as partial credit.
   - Parameters: question (this is "Q" followed by the number of the question. For example, question 10 is labelled as Q10) and response (the student responses to the questions, pulled from the CSV file).



In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets

# The Following Code Removes the Non-Consent and Non-Finished Rows (Cleaning Data)

In [2]:
df = pd.read_excel('/Users/akshaykelshiker/Downloads/Med student pre assessment 2.8.23.xlsx')

df = df.drop([0, 1])

df = df[(df["Finished"] == True)]

df = df[(df["Q52"] != 'Click here if you decline to participate in the research study')]

df_q_only = df.iloc[:, 33:]

df_q_only = df_q_only.drop(["Q10", "Q30", "Q11","Q53_1","Q53_2","Q53_3","Q53_4","Q53_5","SC0","Unnamed: 48"], axis=1)

df_q_only.tail()




,Q6,Q31,Q34,Q36,Q37,Q39,Q12,Q13,Q22,Q23
76,"Anxiety, Depression, & Trauma-Related Disorders",Medical providers may need to engage family me...,"Pain (ex: dental, ear),Cardiovascular disease,...","Constipation, dental pain and gastroesphogeal ...",Talking to inanimate objects,Auditory hallucinations that are intrusive and...,"Ask your patient exploratory questions,Ask que...",Get a copy of their behavior plan for the next...,"Combination treatment, including pharmacothera...",lithium
77,"Anxiety, Depression, & Trauma-Related Disorders","In order to receive IDD services, the person m...","Neurological disorder(s),Asthma,Pain (ex: dent...","Constipation, dental pain and gastroesphogeal ...",Increased repetitive speech and movement,Unusual interests,Ask your patient exploratory questions,Get a copy of their behavior plan for the next...,An extensive review of previous diagnoses and ...,divalproex
78,"Personality Disorders, Psychotic Disorders & T...",The person’s case manager/service coordinator ...,Asthma,Medical/biological issues should be ruled out ...,Running away,"talking to themselves, monologue,having an ima...","Ask your patient exploratory questions,Talk to...","Discipline the person to get them to behave,Ge...","Extensive, long-term behavior modification tre...","lithium,quetiapine,divalproex"
79,"Depression, Psychotic Disorders, & Schizophrenia",The majority of people with IDD receive care i...,Asthma,"Constipation, dental pain and gastroesphogeal ...",Withdrawal,Lip smacking,Know that people with the same disability will...,Get a copy of their behavior plan for the next...,"Extensive, long-term behavior modification tre...",lithium
101,"Anxiety, Depression, & Trauma-Related Disorders","In order to receive IDD services, the person m...","Neurological disorder(s),Cardiovascular diseas...",Medical/biological issues should be ruled out ...,Staring into space,Auditory hallucinations that are intrusive and...,"Ask your patient exploratory questions,Ask que...","Defer to the person’s caregiver,Give the patie...",An extensive review of previous diagnoses and ...,"quetiapine,aripiprazole,risperidone"


In [3]:
questions_dict = {
    'Q6' : {
    'question' : 'The most common mental health conditions experienced by people with intellectual and developmental disabilities are:',
    'choices' : ['Personality Disorders, Psychotic Disorders & Trauma-Related Disorders','Anxiety, Depression, & Trauma-Related Disorders','Bipolar Disorder, Psychotic Disorders, & Trauma-Related Disorders','Depression, Psychotic Disorders & Schizophrenia'],
    'answer(s)' : ['Anxiety, Depression, & Trauma-Related Disorders']  
    },
    
    'Q31' : {
    'question' : 'Which of the following statements are true about systems of care for people with intellectual and developmental disabilities? Select all correct answers.',
    'choices' : ['In order to receive IDD services, the person must have a diagnosed intellectual or developmental disability, and must have impaired adaptive functioning','Medical providers may need to engage family members and other caregivers in the patient’s care','All people with IDD receive the same array of services', 'The majority of people with IDD receive care in institutions','The person’s case manager/service coordinator from the developmental disabilities service agency could be helpful at medical appointments'],
    'answer(s)' : ['In order to receive IDD services, the person must have a diagnosed intellectual or developmental disability, and must have impaired adaptive functioning',"Medical providers may need to engage family members and other caregivers in the patient's care",'The person’s case manager/service coordinator from the developmental disabilities service agency could be helpful at medical appointments']
    },
    
    'Q34' : {
    'question' :'What are the three (3) most common medical conditions experienced by people with IDD?',
    'choices' : ['Neurological disorder(s)','Leg pain', 'Asthma','Pain (ex:dental, ear)','Kidney Disease','Cardiovascular disease','GI problems (ex: constipation, GERD)'],
    'answer(s)' : ['Neurological disorder(s)','Pain (ex: dental, ear)','GI problems (ex: constipation, GERD)']

    },
    
    'Q36' : {
    'question' :'Which of the following statements are true when assessing aggression in patients with intellectual and developmental disabilities? Select all correct answers.',
    'choices' : ['Medical/biological issues should be ruled out first','Some genetic disorders can cause increased risk of mental health or medical issues','Constipation, dental pain and gastroesphogeal reflux are common medical causes of aggression that should be considered',"Aggression should be managed by your state's developmental disabilities service agency", "Behavior is a form of communication",'Psychosocial stressors don\'t need to be considered'],
    'answer(s)' : ['Medical/biological issues should be ruled out first','Some genetic disorders can cause increased risk of mental health or medical issues']
    },
    
    
    'Q37' : {
    'question' :'Which of the following is not a common presentation of anxiety in people with intellectual and developmental disabilities and limited communication skills?',
    'choices' : ['Irritability or aggression','Running away','Talking to inanimate objects','Withdrawal','Staring into space', 'Increased repetitive speech and movement'],
    'answer(s)' : ['Talking to inanimate objects']
    },
    
    'Q39' : {
    'question' : 'Signs of psychosis in people with intellectual and developmental disabilities include which of the following? Select all correct answers.',
    'choices' : ['talking to themselves, monologue','having an imaginary friend','Unusual interests', 'Lip smacking','Auditory hallucinations that are intrusive and distressing','Change from baseline behavior','Covering ears/eyes'],
    'answer(s)' : ['Auditory hallucinations that are intrusive and distressing','Change from baseline behavior','Covering ears/eyes']
    },
    
    
    'Q12' : {
    'question' : 'Which of the following are best practices in communicating with patients with IDD-MH and their families? Select all correct answers.',
    'choices' : ['Ask your patient exploratory questions','Talk to, and engage with, the patients’ caregiver rather than the patient themselves','Ask questions other than that directly pertain to the reason for the visit', 'Know that people with the same disability will always communicate in the same way'],
    'answer(s)' : ['Ask your patient exploratory questions','Ask questions other than that directly pertain to the reason for the visit']
    },
    
    'Q13' : {
    'question' : 'If a patient with IDD-MH is acting out in the waiting room, you and your waiting room staff should: Select all correct answers.',
    'choices' : ['Discipline the person to get them to behave','Get a copy of their behavior plan for the next time they are in your office','Administer a PRN',"Defer to the person's caregiver", 'Give the patient the option to wait outside or in the car'],
    'answer(s)' : ['Defer to the person’s caregiver']
    },
    
    'Q22' : {
    'question' : 'Which is not a treatment approach that people diagnosed with intellectual and developmental disabilities and OCD may need?',
    'choices' : ['An extensive review of previous diagnoses and treatment','Extensive, long-term behavior modification treatment','Combination treatment, including pharmacotherapy/manualized psychotherapies'],
    'answer(s)' : ['Combination treatment, including pharmacotherapy/manualized psychotherapies']
    },
    
    'Q23' : {
    'question' : 'Which of the medications below are among those that should be considered first when treating patients with intellectual and developmental disabilities and mania?Select all correct answers.',
    'choices' : ['lithium','divalproex'],
    'answer(s)' : ['lithium','quetiapine','divalproex','asenapine','aripiprazole','paliperidone','risperidone','cariprazine']               
    }
}

In [4]:
def regrading(question, student_response):
    corr_answer = []
    
    for i in range(len(questions_dict[question]['answer(s)'])):
        corr_answer.append(questions_dict[question]["answer(s)"][i])

    num_correct = len(corr_answer)
    num_common = len(set(corr_answer) & set(student_response))
    num_response = len(student_response)

    if len(questions_dict[question]['answer(s)']) == 1:
        return(int(student_response == corr_answer))
    
    # more than one answer
    else: 
        if num_correct == num_common & num_correct == num_response: # full credit
            return 1.0
 
        elif num_common > 0:
            return num_common/num_correct
            
        else: # totally wrong! 
            return 0.0


In [5]:
single_answer_columns = ["Q6", "Q37", "Q13", "Q22"]

unwanted_substrings = ['talking to themselves', 'monologue']

scores_list = []

for col in df_q_only.columns:
    for response in df_q_only[col]:
        if col in single_answer_columns:
            student_response = [response]
            score = regrading(col, student_response)
            scores_list.append(score)

        else: 
            response = [s.strip() for s in response.split(',') if s.strip() not in unwanted_substrings]
            student_response = response
            score = regrading(col, student_response)
            scores_list.append(score)


In [8]:
q6_answers = []
q31_answers = []
q34_answers = []
q36_answers = []
q37_answers = []
q39_answers = []
q12_answers = []
q13_answers = []
q22_answers = []
q23_answers = []

for i, score in enumerate(scores_list):
    if i < 79:
        q6_answers.append(score)
    elif i < 158:
        q31_answers.append(score)
    elif i < 237:
        q34_answers.append(score)
    elif i < 316:
        q36_answers.append(score)
    elif i < 395:
        q37_answers.append(score)
    elif i < 474:
        q39_answers.append(score)
    elif i < 553:
        q12_answers.append(score)
    elif i < 632:
        q13_answers.append(score)
    elif i < 711:
        q22_answers.append(score)
    else:
        q23_answers.append(score)

scores_df = pd.DataFrame()

    #columns = ["q6_answers", "q31_answers","q34_answers","q36_answers",
       # "q37_answers","q39_answers","q12_answers","q13_answers","q22_answers","q23_answers"])

scores_df['q6_answers'] = q6_answers 
# scores_df['q31_answers'] = q31_answers
# scores_df['q34_answers'] = q34_answers
scores_df['q36_answers'] = q36_answers
scores_df['q37_answers'] = q37_answers
scores_df['q39_answers'] = q39_answers
scores_df['q12_answers'] = q12_answers
scores_df['q13_answers'] = q13_answers
scores_df['q22_answers'] = q22_answers
scores_df['q23_answers'] = q23_answers
scores_df.head(n=10)

,q6_answers,q36_answers,q37_answers,q39_answers,q12_answers,q13_answers,q22_answers,q23_answers
0,1,1.0,0,1.000000,1.0,0,0,0.625
1,1,1.0,0,0.666667,1.0,0,0,0.125
2,1,1.0,0,1.000000,1.0,0,0,0.250
3,1,1.0,1,1.000000,0.5,0,0,0.875
4,1,1.0,1,0.666667,0.5,0,0,0.250
5,1,1.0,0,1.000000,1.0,0,0,0.250
6,0,0.5,0,1.000000,1.0,0,0,0.500
7,1,1.0,0,1.000000,1.0,0,1,0.375
8,1,1.0,1,1.000000,1.0,0,0,0.375
9,1,1.0,1,0.666667,0.5,0,1,0.500


In [9]:
q31_manual = [1,1,1,1,1,1,1,1,1,1,0.66,1,1,1,1,0.33,1,1,0.66,1,1,1,1,0.33,1,0.66,1,1,1,1,1,
             0.33,1,1,0.33,1,0.66,0.66,1,1,1,1,1,1,1,0.66,0.33,1,0.33,0.66,1,0.66,1,0.66,0.33,
             0.33,0.66,0.33,0.66,1,1,0.66,1,0.66,1,0.66,0.66,0,0.66,0.66,0.33,1,0.33,0.33,1,0.33,0.33,0.33,0]
scores_df['q31_answers'] = q31_manual

q34_manual = [1,0.66,1,1,0.66,0.66,0.66,0.66,1,0.66,0.66,0.66,1,1,0.66,1,0.66,1,1,1,0.66,0.66,
             1,0.33,1,0.33,0.66,0.66,1,0.66,0.66,0.66,0.66,0.66,0.66,0.66,0.66,0.66,1,1,1,0.33,0.66,0.66,0.66,
              1,1,0.66,0.66,1,0.66,0.66,0.66,0.66,0.66,0.66,0.66,0.66,0.33,0.66,0.66,1,0.33,0.33,1,
              0.66,0.66,1,1,0.66,0.33,0.66,0.33,0.66,0.33,0.66,0.66,0,0]
scores_df['q34_answers'] = q34_manual
scores_df

scores_df.to_csv("scores_df.csv")